# QGS model: Simple run example with computation of the Lyapunov vectors (see last section)

## Reinhold and Pierrehumbert 1982 model version

This model version is a simple 2-layer channel QG atmosphere truncated at wavenumber 2 on a beta-plane with a simple orography (a montain and a valley). 

More detail can be found in the articles:
* Reinhold, B. B., & Pierrehumbert, R. T. (1982). *Dynamics of weather regimes: Quasi-stationary waves and blocking*. Monthly Weather Review, **110** (9), 1105-1145. [doi:10.1175/1520-0493(1982)110%3C1105:DOWRQS%3E2.0.CO;2](https://doi.org/10.1175/1520-0493(1982)110%3C1105:DOWRQS%3E2.0.CO;2)
* Cehelsky, P., & Tung, K. K. (1987). *Theories of multiple equilibria and weather regimes—A critical reexamination. Part II: Baroclinic two-layer models*. Journal of the atmospheric sciences, **44** (21), 3282-3303. [doi:10.1175/1520-0469(1987)044%3C3282%3ATOMEAW%3E2.0.CO%3B2](https://doi.org/10.1175/1520-0469(1987)044%3C3282%3ATOMEAW%3E2.0.CO%3B2)

## Modules import

First, setting the path and loading of some modules

In [ ]:
import sys, os

In [ ]:
sys.path.extend([os.path.abspath('../../')])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
from matplotlib import rc
rc('font',**{'family':'serif','sans-serif':['Times'],'size':14})

Initializing the random number generator (for reproducibility). -- Disable if needed.

In [ ]:
np.random.seed(210217)

Importing the model's modules

In [ ]:
from qgs.params.params import QgParams
from qgs.integrators.integrator import RungeKuttaIntegrator
from qgs.functions.tendencies import create_tendencies
from qgs.plotting.util import std_plot

Importing the Lyapunovs Estimators

In [ ]:
from qgs.toolbox.lyapunov import LyapunovsEstimator, CovariantLyapunovsEstimator

and diagnostics

In [ ]:
from qgs.diagnostics.streamfunctions import MiddleAtmosphericStreamfunctionDiagnostic
from qgs.diagnostics.variables import VariablesDiagnostic
from qgs.diagnostics.multi import MultiDiagnostic, FieldsDiagnosticsList

## Systems definition

General parameters

In [ ]:
# Time parameters
dt = 0.1
# Saving the model state n steps
write_steps = 5

number_of_trajectories = 1
number_of_perturbed_trajectories = 10

Setting some model parameters

In [ ]:
# Model parameters instantiation with some non-default specs
model_parameters = QgParams({'phi0_npi': np.deg2rad(50.)/np.pi, 'hd':0.3})
# Mode truncation at the wavenumber 2 in both x and y spatial coordinate
model_parameters.set_atmospheric_channel_fourier_modes(2, 2)

# Changing (increasing) the orography depth and the meridional temperature gradient
model_parameters.ground_params.set_orography(0.4, 1)
model_parameters.atemperature_params.set_thetas(0.2, 0)

In [ ]:
# Printing the model's parameters
model_parameters.print_params()

Creating the tendencies function

In [ ]:
f, Df = create_tendencies(model_parameters)

## Time integration

Defining an integrator

In [ ]:
integrator = RungeKuttaIntegrator()
integrator.set_func(f)

Start on a random initial condition and integrate over a transient time to obtain an initial condition on the attractors

In [ ]:
%%time
ic = np.random.rand(model_parameters.ndim)*0.1
integrator.integrate(0., 200000., dt, ic=ic, write_steps=0)
time, ic = integrator.get_trajectories()

Now integrate to obtain a trajectory on the attractor

In [ ]:
%%time
integrator.integrate(0., 100000., dt, ic=ic, write_steps=write_steps)
reference_time, reference_traj = integrator.get_trajectories()

In [ ]:
varx = 0
vary = 1
varz = 2

fig = plt.figure(figsize=(10, 8))
axi = fig.add_subplot(111, projection='3d')

axi.scatter(reference_traj[varx], reference_traj[vary], reference_traj[varz], s=0.2);

axi.set_xlabel('$'+model_parameters.latex_var_string[varx]+'$')
axi.set_ylabel('$'+model_parameters.latex_var_string[vary]+'$')
axi.set_zlabel('$'+model_parameters.latex_var_string[varz]+'$');

In [ ]:
varx = 2
vary = 1
plt.figure(figsize=(10, 8))

plt.plot(reference_traj[varx], reference_traj[vary], marker='o', ms=0.07, ls='')

plt.xlabel('$'+model_parameters.latex_var_string[varx]+'$')
plt.ylabel('$'+model_parameters.latex_var_string[vary]+'$');

In [ ]:
var = 1
plt.figure(figsize=(10, 8))

plt.plot(model_parameters.dimensional_time*reference_time, reference_traj[var])

plt.xlabel('time (days)')
plt.ylabel('$'+model_parameters.latex_var_string[var]+'$');

## Lyapunov exponents and vectors estimation example

Backward Lyapunovs Estimation

In [ ]:
lvint = LyapunovsEstimator()

In [ ]:
%%time

lvint.set_func(f, Df)
lvint.compute_lyapunovs(0., 10000., 20000., 0.1, 0.1, ic, write_steps=1)
btl, btraj, bexp, bvec = lvint.get_lyapunovs()


In [ ]:
plt.figure(figsize=(15, 4))

mean_exp = np.mean(bexp, axis=-1)

x_pos = np.arange(1.,model_parameters.ndim+1,1)

plt.bar(x_pos, mean_exp)

plt.vlines(x_pos, -0.55, np.minimum(0.,mean_exp)-0.035, linestyles='dashdot', colors='tab:gray')

plt.xticks(x_pos, map(str,range(1, model_parameters.ndim+1,1)))
yt=[-0.5,-0.4,-0.3,-0.2,-0.1,0.,0.1]
plt.yticks(yt, map(str,yt))

plt.xlim(x_pos[0]-1., x_pos[-1]+1.)
plt.ylim(np.min(mean_exp)-0.1, np.max(mean_exp)+0.1)

plt.ylabel("Lyapunov exponent");
plt.xlabel("Index of the Lyapunov exponent");


In [ ]:
plt.figure(figsize=(10, 8))

plt.plot(model_parameters.dimensional_time*btl, bexp.T)

plt.xlabel('time (days)')

plt.xlim(1200., 1220.)

plt.title('Local Backward Lyapunov exponents');

In [ ]:
plt.figure(figsize=(10, 8))

plt.plot(model_parameters.dimensional_time*btl, bvec[:, 0, :].T)

plt.xlabel('time (days)')

plt.xlim(1200., 1220.)

plt.title('Most unstable Backward Lyapunov vector components');

Forward Lyapunovs Estimation

In [ ]:
%%time

lvint.compute_lyapunovs(0., 10000., 20000., 0.1, 0.1, ic, write_steps=1, forward=True)
ftl, ftraj, fexp, fvec = lvint.get_lyapunovs()


In [ ]:
plt.figure(figsize=(15, 4))

mean_exp = np.mean(fexp, axis=-1)[::-1]

x_pos = np.arange(1.,model_parameters.ndim+1,1)

plt.bar(x_pos, mean_exp)

plt.vlines(x_pos, -0.55, np.minimum(0.,mean_exp)-0.035, linestyles='dashdot', colors='tab:gray')

plt.xticks(x_pos, map(str,range(1, model_parameters.ndim+1,1)))
yt=[-0.5,-0.4,-0.3,-0.2,-0.1,0.,0.1]
plt.yticks(yt, map(str,yt))

plt.xlim(x_pos[0]-1., x_pos[-1]+1.)
plt.ylim(np.min(mean_exp)-0.1, np.max(mean_exp)+0.1)

plt.ylabel("Lyapunov exponent");
plt.xlabel("Index of the Lyapunov exponent");


In [ ]:
plt.figure(figsize=(10, 8))

plt.plot(model_parameters.dimensional_time*ftl, fexp.T)

plt.xlabel('time (days)')

plt.xlim(200., 220.)

plt.title('Local Forward Lyapunov exponents');

In [ ]:
plt.figure(figsize=(10, 8))

plt.plot(model_parameters.dimensional_time*ftl, fvec[:, -1, :].T)

plt.xlabel('time (days)')
plt.ylabel('Amplitude')

plt.xlim(200., 220.)

plt.title('Most unstable Lyapunov vector components');

In [ ]:
lvint.terminate()

Covariant Lyapunovs Estimation

In [ ]:
clvint = CovariantLyapunovsEstimator()

With the Ginelli et al. algorithm

In [ ]:
%%time

clvint.set_func(f, Df)
clvint.compute_clvs(0., 10000., 20000., 30000., 0.1, 0.1, ic, write_steps=1)
ctl, ctraj, cexp, cvec = clvint.get_clvs()


In [ ]:
plt.figure(figsize=(15, 4))

mean_exp = np.mean(cexp, axis=-1)

x_pos = np.arange(1.,model_parameters.ndim+1,1)

plt.bar(x_pos, mean_exp)

plt.vlines(x_pos, -0.55, np.minimum(0.,mean_exp)-0.035, linestyles='dashdot', colors='tab:gray')

plt.xticks(x_pos, map(str,range(1, model_parameters.ndim+1,1)))
yt=[-0.5,-0.4,-0.3,-0.2,-0.1,0.,0.1]
plt.yticks(yt, map(str,yt))

plt.xlim(x_pos[0]-1., x_pos[-1]+1.)
plt.ylim(np.min(mean_exp)-0.1, np.max(mean_exp)+0.1)

plt.ylabel("Lyapunov exponent");
plt.xlabel("Index of the Lyapunov exponent");


In [ ]:
plt.figure(figsize=(10, 8))

plt.plot(model_parameters.dimensional_time*ctl, cexp.T)

plt.xlabel('time (days)')

plt.xlim(1200., 1220.)

plt.title('Local Lyapunov exponents');

In [ ]:
plt.figure(figsize=(10, 8))

plt.plot(model_parameters.dimensional_time*ctl, cvec[:, 0, :].T)

plt.xlabel('time (days)')

plt.xlim(1200., 1220.)

plt.title('Most unstable Lyapunov vector components');

With the intersection of the Osedelet subspace algorithm

In [ ]:
%%time

clvint.compute_clvs(0., 10000., 20000., 30000., 0.1, 0.1, ic, write_steps=10, method=1)
ctl_sub, ctraj_sub, cexp_sub, cvec_sub = clvint.get_clvs()


In [ ]:
plt.figure(figsize=(15, 4))

mean_exp = np.mean(cexp_sub, axis=-1)

x_pos = np.arange(1.,model_parameters.ndim+1,1)

plt.bar(x_pos, mean_exp)

plt.vlines(x_pos, -0.55, np.minimum(0.,mean_exp)-0.035, linestyles='dashdot', colors='tab:gray')

plt.xticks(x_pos, map(str,range(1, model_parameters.ndim+1,1)))
yt=[-0.5,-0.4,-0.3,-0.2,-0.1,0.,0.1]
plt.yticks(yt, map(str,yt))

plt.xlim(x_pos[0]-1., x_pos[-1]+1.)
plt.ylim(np.min(mean_exp)-0.1, np.max(mean_exp)+0.1)

plt.ylabel("Lyapunov exponent");
plt.xlabel("Index of the Lyapunov exponent");


In [ ]:
plt.figure(figsize=(10, 8))

plt.plot(model_parameters.dimensional_time*ctl_sub, cexp_sub.T)

plt.xlabel('time (days)')
plt.ylabel('Amplitude')

plt.xlim(1200., 1220.)

plt.title('Local Lyapunov exponents');

In [ ]:
plt.figure(figsize=(10, 8))

plt.plot(model_parameters.dimensional_time*ctl_sub, cvec_sub[:, 0, :].T)

plt.xlabel('time (days)')
plt.ylabel('Amplitude')

plt.xlim(1200., 1220.)

plt.title('Most unstable Lyapunov vector components');

In [ ]:
clvint.terminate()

## Showing the resulting fields (animation)

This is an advanced feature showing the time evolution of diagnostic of the model. It shows simultaneously a scatter plot of the variable $\psi_2$ and $\psi_3$, and the geopotential height field at 500 hPa over the orographic height. Please read the documentation for more information.

Creating the diagnostics:

* For the 500hPa geopotential height:

In [ ]:
psi = MiddleAtmosphericStreamfunctionDiagnostic(model_parameters, geopotential=True)

* For the first covariant Lyapunov vector (components of the 500hPa geopotential height)

In [ ]:
psi_clyap = MiddleAtmosphericStreamfunctionDiagnostic(model_parameters, geopotential=True)

* For the nondimensional variables $\psi_2$ and $\psi_3$:

In [ ]:
variable_nondim = VariablesDiagnostic([2, 1], model_parameters, False)

In [ ]:
# setting also the background
background = VariablesDiagnostic([2, 1], model_parameters, False)
background.set_data(reference_time, reference_traj)

Loading the data in a combined diagnostic (FieldsDiagnosticsList)

In [ ]:
psi.set_data(ctl, ctraj)
psi_clyap.set_data(ctl, cvec[:,0,:])

lyap_diags = FieldsDiagnosticsList([psi, psi_clyap])

Creating a multi diagnostic with both a scatter plot and the combined diagnostics:

In [ ]:
m = MultiDiagnostic(1,2)
m.add_diagnostic(variable_nondim, 
                 diagnostic_kwargs={'show_time': False, 'background': background}, 
                 plot_kwargs={'ms': 0.2})
m.add_diagnostic(lyap_diags, 
                 diagnostic_kwargs={'style': ['image', 'contour'],
                                    'contour_labels': False,
                                    'show_time': True,
                                    'oro_kwargs': False}, 
                 plot_kwargs=[{'cmap': plt.get_cmap('gist_yarg')}, {'colors':'k'}])
m.set_data(ctl, ctraj)

and show an interactive animation (with the first covariant Lyapunov vector as contour plot):

In [ ]:
rc('font',**{'family':'serif','sans-serif':['Times'],'size':12})
m.animate(figsize=(20,6))

or a movie:

In [ ]:
rc('font',**{'family': 'serif','sans-serif': ['Times'],'size': 12})
m.movie(figsize=(20,6), anim_kwargs={'interval': 100, 'frames': 1000})